In [1]:
Example_out = {
  "paperId": "649def34f8be52c8b66281af98ae884c09aef38b",
  "externalIds": {
    "ArXiv": "...",
    "DBLP": "...",
    "PubMedCentral": "..."
  },
  "url": "https://www.semanticscholar.org/paper/649def34f8be52c8b66281af98ae884c09aef38b",
  "title": "Construction of the Literature Graph in Semantic Scholar",
  "abstract": "We describe a deployed scalable system for organizing published scientific literature into a heterogeneous graph to facilitate algorithmic manipulation and discovery.",
  "venue": "International Conference on Software Engineering",
  "publicationVenue": {
    "id": "a36dc29e-4ea1-4567-b0fe-1c06daf8bee8",
    "name": "International Conference on Software Engineering",
    "type": "conference",
    "alternate_names": [
      "IEEE Int Conf Semicond Electron",
      "IEEE International Conference on Semiconductor Electronics",
      "ICSE",
      "Int Conf Softw Eng"
    ],
    "url": "http://www.icse-conferences.org/"
  },
  "year": 2018,
  "referenceCount": 321,
  "citationCount": 987,
  "influentialCitationCount": 654,
  "isOpenAccess": True,
  "openAccessPdf": {
    "url": "https://www.aclweb.org/anthology/N18-3011.pdf",
    "status": "HYBRID"
  },
  "fieldsOfStudy": [
    "Computer Science"
  ],
  "s2FieldsOfStudy": [
    {
      "category": "Computer Science",
      "source": "external"
    },
    {
      "category": "Computer Science",
      "source": "s2-fos-model"
    },
    {
      "category": "Mathematics",
      "source": "s2-fos-model"
    }
  ],
  "publicationTypes": [
    "Journal Article",
    "Review"
  ],
  "publicationDate": "2015-01-17",
  "journal": {
    "name": "Remote Sensing of Environment",
    "pages": "255-271",
    "volume": "176"
  },
  "citationStyles": {
    "bibtex": "@['JournalArticle', 'Conference']{Ammar2018ConstructionOT,\n author = {Waleed Ammar and Dirk Groeneveld and Chandra Bhagavatula and Iz Beltagy and Miles Crawford and Doug Downey and Jason Dunkelberger and Ahmed Elgohary and Sergey Feldman and Vu A. Ha and Rodney Michael Kinney and Sebastian Kohlmeier and Kyle Lo and Tyler C. Murray and Hsu-Han Ooi and Matthew E. Peters and Joanna L. Power and Sam Skjonsberg and Lucy Lu Wang and Christopher Wilhelm and Zheng Yuan and Madeleine van Zuylen and Oren Etzioni},\n booktitle = {NAACL},\n pages = {84-91},\n title = {Construction of the Literature Graph in Semantic Scholar},\n year = {2018}\n}\n"
  },
  "authors": [
    {
      "authorId": "1741101",
      "name": "Oren Etzioni"
    }
  ]
}

### Get papers for every id:

In [12]:
import pandas as pd
ids = open("data/IDs_real.txt").read().splitlines()
print(len(ids))
id_groups = [ids[x:x+50] for x in range(0, len(ids), 50)]
print(len(id_groups))
print(sum([len(lst) for lst in id_groups]))

2057
42
2057


In [13]:
import requests
BASE_URL = "https://api.semanticscholar.org/graph/"
VERSION = "v1/"
RESOURCE = "author/batch"
URL = BASE_URL + VERSION + RESOURCE

# params2 = {"ids": ids_groups[0][:5]}

params = {"fields": "authorId,name,aliases,citationCount,papers.fieldsOfStudy,papers.authors,papers.title,papers.year,papers.externalIds,papers.citationCount,papers.abstract"}

In [18]:
content = []

In [19]:
for i in range(len(id_groups)):
    print(i)
    json_data = {"ids": id_groups[i]}
    r = requests.post(URL, params = params, json = json_data)
    current_content = r.json()
    content += current_content

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41


In [28]:
print(*content[0]["papers"], sep="\n\n")

{'paperId': 'cd29131082e7cc7f4d9c333afed8b58d4cf44ce5', 'externalIds': {'DOI': '10.1109/SPMB52430.2021.9672258', 'CorpusId': 246039450}, 'title': 'Feature Reconstruction Based Channel Selection for Emotion Recognition Using EEG', 'abstract': 'There has been a surge in the use of consumer grade wearable Electroencephalogram (EEG) devices for emotion discrimination tasks in various research laboratories in recent times. The obvious advantages carried by these compared to medical grade equipment are reduced costs and portability, which enable monitoring for a longer term and in more natural environment. Different manufacturers of consumer grade EEG devices place the electrodes at different locations. In this paper we present a novel method for determining locations of the fewest electrodes with the most emotion valence discriminative power. It starts with feature generation and selection for identifying positional features for the classification task, followed by channel selection that mi

### Finding neighbors

In [30]:
neighbors = []
for i in range(len(content)):
    authorDict = content[i]
    if authorDict == "message":
        print(i, " message"); continue
    try: paperDicts = authorDict["papers"]
    except: print(i, " paperdict"); continue
    for paperDict in paperDicts:
        neighbors += [author["authorId"] for author in paperDict["authors"]]

435  paperdict
800  message
1023  paperdict
1841  paperdict


In [32]:
true_neighbor_ids = set(neighbors + ids) - set(ids)
len(true_neighbor_ids), list(true_neighbor_ids)[:10]

(108246,
 ['2112868919',
  '1783281',
  '114885129',
  '48164318',
  '2064494954',
  '2006291',
  '101662683',
  '2069663304',
  '2152605987',
  '143943324'])

#### Save neighbors

In [33]:
neighbor_groups = [list(true_neighbor_ids)[x:x+50] for x in range(0, len(true_neighbor_ids), 50)]
neighbor_content = []
print(len(neighbor_groups))

2165


In [1]:
print(len(neighbor_content))

NameError: name 'neighbor_content' is not defined

In [ ]:
import time
start_time = time.time()

for i in range(len(neighbor_groups)):
    json_data = {"ids": neighbor_groups[i]}
    r = requests.post(URL, params = params, json = json_data)
    print(i)
    current_content = r.json()
    neighbor_content += current_content

### Creating author dataset

In [43]:
content = neighbor_content

In [45]:
header = ["AuthorID", "Name", "Aliases", "Citation Count", "Field"]
author_dataset = []
skipped = 0
for authorDict in content:
    current_list = []
    if authorDict == "message":
        skipped += 1
        print(skipped)
        continue
    current_list.append(authorDict["authorId"])
    current_list.append(authorDict["name"])
    current_list.append(authorDict["aliases"])
    current_list.append(authorDict["citationCount"])
    if len([str(paper["fieldsOfStudy"]) for paper in authorDict["papers"]]) > 1:
        current_list.append(max([str(paper["fieldsOfStudy"]) for paper in authorDict["papers"]]))
    else:
        current_list.append("None")
    author_dataset.append(current_list)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [46]:
author_dataframe = pd.DataFrame(data=author_dataset, columns=header)
author_dataframe

,AuthorID,Name,Aliases,Citation Count,Field
0,3751805,B. Guerci,"[B Guerci, B. Guerci, Bruno Guerci]",7506,['Psychology']
1,3798644,R. Kerby,"[R Kerby, Robert L. Kerby, R. L. Kerby, R L Ke...",4197,['Sociology']
2,113453486,B. Tana,"[B. H. Tana, Bo Tana, Bowling Tana]",0,['Biology']
3,2089311,Eileen F. Tallman,"[Eileen F Tallman, Eileen Frances Tallman, Eil...",534,['Psychology']
4,3128414,E. Ritacco,"[E. Ritacco, Ettore Ritacco]",364,['Computer Science']
...,...,...,...,...,...
2144,15985480,R. Dragoi,"[R Dragoi, R. Dragoi, Ruxandra Dragoi]",42,['Medicine']
2145,65985240,G. Alleva,"[G. Alleva, Giorgio Alleva]",545,['Psychology']
2146,3239668,F. Morone,"[F Morone, Flavia Cristina Morone, Flaviano Mo...",1865,"['Psychology', 'Medicine']"
2147,2227410,Yueshen Xu,[Yueshen Xu],1553,"['Medicine', 'Computer Science']"


### Creating paper dataset

In [267]:
content[0]["papers"][0]["externalIds"]["DOI"]

'10.1109/SPMB52430.2021.9672258'

In [268]:
header_paper = ["paperId", "title", "year", "externalId.DOI", "Citation Count", "fields", "authorIds", "authorNames"]
paper_dataset = []
for authorDict in content:
    paperDicts = authorDict["papers"]
    for paperDict in paperDicts:
        current_list = []
        current_list.append(paperDict["paperId"])
        current_list.append(paperDict["title"])
        current_list.append(paperDict["year"])
        if "DOI" in paperDict["externalIds"].keys():
            current_list.append(paperDict["externalIds"]["DOI"])
        else:
            current_list.append("None")
        current_list.append(paperDict["citationCount"])
        current_list.append(paperDict["fieldsOfStudy"])
        current_list.append([author["authorId"] for author in paperDict["authors"]])
        current_list.append([author["name"] for author in paperDict["authors"]])
        paper_dataset.append(current_list)
    

In [269]:
paper_dataframe = pd.DataFrame(data=paper_dataset, columns=header_paper)
paper_dataframe

,paperId,title,year,externalId.DOI,Citation Count,fields,authorIds,authorNames
0,cd29131082e7cc7f4d9c333afed8b58d4cf44ce5,Feature Reconstruction Based Channel Selection...,2021.0,10.1109/SPMB52430.2021.9672258,1,None,"[2150507237, 2107389568, 2110194494]","[J. R. Msonda, Z. He, C. Lu]"
1,33b43646408161ad66a80ba8079d6e1aee244b65,Wavelet multipliers and signals,1999.0,10.1017/S0334270000010523,38,[Mathematics],"[2107389568, 11214739]","[Z. He, M. W. Wong]"
2,3bda43967a06cf4611c5e5c8636477c230663201,Principal Component Feature for Speech Recogni...,1999.0,None,1,[Mathematics],[2107389568],[Z. He]
3,3972ac23a6eaf45dea73cab154706088498018b8,Chasm in Hegemony: Explaining and Reproducing ...,2021.0,10.1145/3460083,3,"[Computer Science, Sociology]","[2108343919, 1643347009, 2051502400, 120446735...","[Yiguang Zhang, Jessy Xinyi Han, Ilica Mahajan..."
4,5dabe81f9d6b53e75b0078322b3f0741f2788041,Chasm in Hegemony,2021.0,10.1145/3543516.3460109,2,[Computer Science],"[2108343919, 1643347009, 2051502400, 120446735...","[Yiguang Zhang, Jessy Xinyi Han, Ilica Mahajan..."
...,...,...,...,...,...,...,...,...
28371,f204e92575b3c37a67a9103d6ed00292578978b3,Mechanism of action of f 3-bungarotoxin on syn...,NaN,None,0,None,"[2113780702, 145366990, 2173858]","[Indira Sen, P. Grantham, J. Cooper]"
28372,13d1782fc7384202e39af2aa9b3b77df2395bfe3,"Risk Analysis of Dam Failure, Floods and Growi...",2018.0,10.2139/ssrn.3254595,0,None,[2059392121],[Binny Mathew]
28373,c633303e309a59baeb0c62e80e22b01f0b44e101,Asura: A Tale of Vanquished,2018.0,10.2139/SSRN.3270075,1,[History],[2059392121],[Binny Mathew]
28374,d2e119f3eab633aabf0e787df3540fa726145e87,Inequality of Gender-Based Victimization In Ma...,2018.0,10.2139/ssrn.3254517,0,[Sociology],[2059392121],[Binny Mathew]


### Paper Abstract Dataset

In [270]:
header_paper_abstract = ["PaperId", "Abstract"]
paper_abstract_dataset = []
for authorDict in content:
    paperDicts = authorDict["papers"]
    for paperDict in paperDicts:
        current_list = []
        current_list.append(paperDict["paperId"])
        current_list.append(paperDict["abstract"])
        paper_abstract_dataset.append(current_list)

In [275]:
paper_abstract_dataframe = pd.DataFrame(data=paper_abstract_dataset, columns=header_paper_abstract)
paper_abstract_dataframe

,PaperId,Abstract
0,cd29131082e7cc7f4d9c333afed8b58d4cf44ce5,There has been a surge in the use of consumer ...
1,33b43646408161ad66a80ba8079d6e1aee244b65,Abstract The Schatten-von Neumann property of ...
2,3bda43967a06cf4611c5e5c8636477c230663201,Using curve fitting and principalcom ponentana...
3,3972ac23a6eaf45dea73cab154706088498018b8,In networks with a minority and a majority com...
4,5dabe81f9d6b53e75b0078322b3f0741f2788041,In networks with a minority and a majority com...
...,...,...
28371,f204e92575b3c37a67a9103d6ed00292578978b3,The neurochemical activity of fl-bungarotoxin ...
28372,13d1782fc7384202e39af2aa9b3b77df2395bfe3,People have exploited water resource to a leve...
28373,c633303e309a59baeb0c62e80e22b01f0b44e101,None
28374,d2e119f3eab633aabf0e787df3540fa726145e87,"Mahesh Dattani being a director, directed many..."


### Save dataframes

In [281]:
author_dataframe.to_csv("data/AuthorDataframe.csv", index=False)
paper_dataframe.to_csv("data/PaperDataset.csv", index=False)
paper_abstract_dataframe.to_csv("data/PaperAbstractDataset.csv", index=False)